## Load env file


In [1]:
from dotenv import load_dotenv

load_dotenv()


True

## Initialize LLM


In [2]:
from crewai import LLM

llm = LLM(model="gemini/gemini-2.0-flash", temperature=0.1)


## Create a tool


In [ ]:
from crewai.tools import BaseTool

sample_original_email = """
looping in Priya. TAS and PRX updates are in the deck. ETA for SDS integration is Friday.
Let's sync up tomorrow if SYNCBOT allows :-) . ping me if any blockers.
"""


class ReplacejargonsTool(BaseTool):
    name: str = "Replace Jargons"
    description: str = " Repl;ace Jargons with specific words"

    def _run(self, email: str) -> str:
        """
        This tool replaces jargons with specific words
        """

        replace_jargons = {
            " :-) ": "smily face",
            "ping me": "reach out to me",
            "wip": "work in progress",
            "ETA": "estimated time of arrival",
            "PRX": "product requirements exchange",
            "TAS": "technical assistance",
            "SDS": "software development standards",
            "SYNCBOT": "syncbot",
            "SYNCBOT": "syncbot",
            "SYNCBOT": "syncbot",
        }
        suggestions = []
        email_lower = email.lower()
        for jargon, replacement in replace_jargons.items():
            if jargon.lower() in email_lower:
                suggestions.append(f"consider replacing {jargon} -> {replacement}")
        return "\n".join(suggestions) if suggestions else "No jargons found"


jt = ReplacejargonsTool()
jt._run(sample_original_email)

'consider replacing  :-)  -> smily face\nconsider replacing ping me -> reach out to me\nconsider replacing ETA -> estimated time of arrival\nconsider replacing PRX -> product requirements exchange\nconsider replacing TAS -> technical assistance\nconsider replacing SDS -> software development standards\nconsider replacing SYNCBOT -> syncbot'

## Create a Agent and Task


In [ ]:
from crewai import Agent, Task, Crew

email_agent = Agent(
    role="Email Agent",
    goal="Send an email to the CEO of the company",
    backstory="You are a helpful assistant that can send emails to the CEO of the company",
    llm=llm,
    verbose=True,
    tools=[jt],
)

sample_original_email = """
hey team, just wanted to tell u that the demo is kind of ready, but there's still stuff left.
Maybe we can show what we have and say rest is WIP.
Let me know what u think. thanks
ping me if any blockers.

PRX and WIP are in the deck.
"""

email_task = Task(
    description=f"""take the following email and rewrite it to professional version. 
    Original email: '''{sample_original_email}'''
    """,
    agent=email_agent,
    expected_output="An email to the CEO of the company, expand the aberrations",
)

email_agent_crew = Crew(agents=[email_agent], tasks=[email_task], verbose=True)

result = email_agent_crew.kickoff()

print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5b96022b-55bd-4c6c-9a25-d29532681f1d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Agent                                                                                             │
│                                                                                                                 │
│  Task: take the following email and rewrite it to professional version.                                         │
│      Original email: '''                                                                                        │
│  hey team, just wanted to tell u that the demo is kind of ready, but there's still stuff left.                  │
│  Maybe we can show what we have and say rest is WIP.                                                            │
│  Let me know what u think. thanks                                                                               │
│  ping me if any blockers.                                                                                       │
│                                                                                                                 │
│  PRX and WIP are in the deck.                                                                                   │
│  '''                                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Agent                                                                                             │
│                                                                                                                 │
│  Thought: I need to rewrite the email to be more professional and suitable for the CEO. This involves           │
│  expanding abbreviations, using formal language, and providing a clear and concise message. I will use the      │
│  "Replace Jargons" tool to achieve this.                                                                        │
│                                                                                                                 │
│  Using Tool: Replace Jargons                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"email\": \"Subject: Demo Readiness Update\\n\\nDear Team,\\n\\nI am writing to provide an update on the    │
│  demo readiness. The demo is nearing completion, but there are still some outstanding items.\\n\\nI propose     │
│  that we present the current version of the demo and clearly state that the remaining features are Work In      │
│  Progress.\\n\\nPlease let me know your thoughts on this approach.\\n\\nKindly inform me if you encounter any   │
│  obstacles.\\n\\n'PRX' and 'WIP' are included in the presentation deck.\\n\\nThank you,\\n[Your Name]\"}"       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Agent                                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Subject: Demo Readiness Update                                                                                 │
│                                                                                                                 │
│  Dear Team,                                                                                                     │
│                                                                                                                 │
│  I am writing to provide an update on the demo readiness. The demo is nearing completion, but there are still   │
│  some outstanding items.                                                                                        │
│                                                                                                                 │
│  I propose that we present the current version of the demo and clearly state that the remaining features are    │
│  Work In Progress.                                                                                              │
│                                                                                                                 │
│  Please let me know your thoughts on this approach.                                                             │
│                                                                                                                 │
│  Kindly inform me if you encounter any obstacles.                                                               │
│                                                                                                                 │
│  'Product Requirements Exchange' and 'Work In Progress' are included in the presentation deck.                  │
│                                                                                                                 │
│  Thank you,                                                                                                     │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0445f87c-07ca-4b16-8c27-f70ad428992e                                                                     │
│  Agent: Email Agent                                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5b96022b-55bd-4c6c-9a25-d29532681f1d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Subject: Demo Readiness Update                                                                   │
│                                                                                                                 │
│  Dear Team,                                                                                                     │
│                                                                                                                 │
│  I am writing to provide an update on the demo readiness. The demo is nearing completion, but there are still   │
│  some outstanding items.                                                                                        │
│                                                                                                                 │
│  I propose that we present the current version of the demo and clearly state that the remaining features are    │
│  Work In Progress.                                                                                              │
│                                                                                                                 │
│  Please let me know your thoughts on this approach.                                                             │
│                                                                                                                 │
│  Kindly inform me if you encounter any obstacles.                                                               │
│                                                                                                                 │
│  'Product Requirements Exchange' and 'Work In Progress' are included in the presentation deck.                  │
│                                                                                                                 │
│  Thank you,                                                                                                     │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Subject: Demo Readiness Update

Dear Team,

I am writing to provide an update on the demo readiness. The demo is nearing completion, but there are still some outstanding items.

I propose that we present the current version of the demo and clearly state that the remaining features are Work In Progress.

Please let me know your thoughts on this approach.

Kindly inform me if you encounter any obstacles.

'Product Requirements Exchange' and 'Work In Progress' are included in the presentation deck.

Thank you,
[Your Name]
